# installing dependencies


In [ ]:
!pip install -U "web3[tester]"
!pip install --force-reinstall protobuf==3.20.3
!pip install --force-reinstall jsonschema==3.2.0
!pip install py-solc-x

# Libraries

In [58]:
from web3 import Web3
from solcx import compile_source
from solcx import install_solc
from eth_tester.exceptions import TransactionFailed

# Configration

In [ ]:
install_solc(version='latest')
w3 = Web3(Web3.EthereumTesterProvider())
w3.eth.default_account= w3.eth.accounts[0] # This will be refered as BB01 the owner of the smart contract

# Smart Contract

## Build Solididty Smart Contract

In [5]:
compiled_solidity = compile_source(
    '''
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.21;

contract BiometricIdentity {

    // Structures
    struct Subject {
        uint16 ID;           // ID of the Subject
        uint16 hx;           // Hash representation
        uint16 delta;        // The offset
    }

    struct Node {
        uint ID;             // ID of the Node
        string name;         // Name of the Node
        address addr;        // Ethereum address of the Node
        bool isAuthorised;   // Authorization status for AC
        bool isEnrollment;   // Enrollment center status for EC
    }

    // Events
    event SubjectSet(uint16 indexed _ID, uint16 _hx, uint16 _delta);
    event SubjectUpdated(uint16 indexed _ID, uint16 _hx, uint16 _delta);
    event SubjectDeleted(uint16 indexed _ID);
    event NodeSet(uint indexed _ID, string _name, address _addr);
    event NodeDeleted(address indexed _addr);
    event NodeStatusUpdated(address indexed _addr, bool isAuthorised, bool isEnrollment);
    event Error(string ErrorMessage);

    // Private state variable for contract creator's address
    address private owner;

    // Mappings
    mapping(uint16 => Subject) private subjects;
    mapping(address => Node) private nodes;

    // Modifiers
    modifier isAC() {
        require(nodes[msg.sender].isAuthorised, "Caller is not an authorized Authentication Center");
        _;
    }

    modifier isEC() {
        require(nodes[msg.sender].isEnrollment, "Caller is not an Enrollment Center");
        _;
    }

    // Constructor
    constructor() {
        owner = msg.sender;
        nodes[owner] = Node({
            ID: 1,
            name: "First Enrollment Center",
            addr: owner,
            isAuthorised: true,
            isEnrollment: true
        });
    }

    // Functions for Subjects
    function setSubject(uint16 _ID, uint16 _hx, uint16 _delta) public isEC {
        subjects[_ID] = Subject(_ID, _hx, _delta);
        emit SubjectSet(_ID, _hx, _delta);
    }

    function getSubject(uint16 _ID) public view isAC returns (uint16, uint16, uint16) {
        Subject memory subject = subjects[_ID];
        return (subject.ID, subject.hx, subject.delta);
    }

    function updateSubject(uint16 _ID, uint16 _hx, uint16 _delta) public isEC {
        Subject storage subject = subjects[_ID];
        subject.hx = _hx;
        subject.delta = _delta;
        emit SubjectUpdated(_ID, _hx, _delta);
    }

    function deleteSubject(uint16 _ID) public isEC {
        delete subjects[_ID];
        emit SubjectDeleted(_ID);
    }

    // Functions for Nodes
    function setNode(uint _ID, string memory _name, address _addr) public isEC {
        nodes[_addr] = Node(_ID, _name, _addr, true, false); // Setting as authorized but not an enrollment center by default
        emit NodeSet(_ID, _name, _addr);
    }

    function getNode(address _addr) public view isEC returns (uint, string memory, address, bool, bool) {
        Node memory node = nodes[_addr];
        return (node.ID, node.name, node.addr, node.isAuthorised, node.isEnrollment);
    }

    function deleteNode(address _addr) public isEC {
        delete nodes[_addr];
        emit NodeDeleted(_addr);
    }

    function updateNodeStatus(address _addr, bool _isAuthorised, bool _isEnrollment) public isEC {
        Node storage node = nodes[_addr];
        node.isAuthorised = _isAuthorised;
        node.isEnrollment = _isEnrollment;
        emit NodeStatusUpdated(_addr, _isAuthorised, _isEnrollment);
    }
}

   '''
   , output_values = ['abi', 'bin']
)

contract_id , contract_interface = compiled_solidity.popitem()
abi = contract_interface['abi']
bytecode = contract_interface['bin']

## Deploy Smart Contract

In [8]:
accessControlContract = w3.eth.contract(address = w3.eth.wait_for_transaction_receipt(w3.eth.contract(abi = abi, bytecode=bytecode).constructor().transact()).contractAddress, abi = abi)

# Testing Smart Contract

In [9]:
accessControlContract.all_functions()

[<Function deleteNode(address)>,
 <Function deleteSubject(uint16)>,
 <Function getNode(address)>,
 <Function getSubject(uint16)>,
 <Function setNode(uint256,string,address)>,
 <Function setSubject(uint16,uint16,uint16)>,
 <Function updateNodeStatus(address,bool,bool)>,
 <Function updateSubject(uint16,uint16,uint16)>]

In [83]:
Main_Enrollment_Center = w3.eth.accounts[0] # Prevously set as default_account and it is the owner of the smart contract # This is an Enrollment Center and an Authentication Center
postOfficeAddress = w3.eth.accounts[1] # Post Office Address for testing as AC node
hospitalAddress = w3.eth.accounts[2] #  Hospital Address for testing as AC node
ComuneAddress  = w3.eth.accounts[3] # Comune Address for testing
whiteHatHackerAddress  = w3.eth.accounts[4] # White-hat Hacker Address for testing as AC node

## Regestration phase

In [84]:
accessControlContract.functions.setNode(1,              # ID
                                        'Post Office',  # Name
                                        postOfficeAddress       # Address
                                        ).transact({'from': Main_Enrollment_Center}) # A transaction from the owner (default_account) of the smart contract

accessControlContract.functions.setNode(2,              # ID
                                        'Hospital',     # Name
                                        hospitalAddress       # Address
                                        ).transact({'from': Main_Enrollment_Center}) # A transaction from the owner (default_account) of the smart contract

HexBytes('0x3a984334bacad11a24ea01634ea3bae8fcca0aeb5c4d3d5dadad2821317a66bc')

## Enrollment phase

In [85]:
accessControlContract.functions.setSubject(1,                         # ID
                                           10110,                     # hx      # Random Number
                                           1001                       # delta   # Random Number
                                           ).transact({'from': Main_Enrollment_Center}) # Enrolled by BB01

accessControlContract.functions.setSubject(2,                         # ID
                                           11110,                     # hx      # Random Number
                                           1101                       # delta   # Random Number
                                           ).transact({'from': Main_Enrollment_Center}) # Enrolled by BB01

accessControlContract.functions.setSubject(3,                         # ID
                                           10111,                     # hx      # Random Numbe
                                           1011                       # delta   # Random Number
                                           ).transact({'from': Main_Enrollment_Center}) # Enrolled by BB01

HexBytes('0xa51ed0c6e9001cfaa35b0d23554b7d392dc3a0eefe393342e2daf4a4638aeb1b')

## Authentication phase

In [86]:
accessControlContract.functions.getSubject(1                                    # ID to be retrived
                                           ).call({'from':Main_Enrollment_Center}) # Call from BB01

[1, 10110, 1001]

In [87]:
accessControlContract.functions.getSubject(2                                    # ID to be retrived
                                           ).call({'from':postOfficeAddress}) # Call from BB02

[2, 11110, 1101]

In [88]:
accessControlContract.functions.getSubject(3                                    # ID to be retrived
                                           ).call({'from':hospitalAddress}) # Call from BB03

[3, 10111, 1011]

# Revocation phase

In [89]:
accessControlContract.functions.deleteSubject(4                                 # Id of the subject to be deleted
                                              ).transact({'from': Main_Enrollment_Center})   # deleter of the subject

HexBytes('0xc5a301d0be6d90c60aedea26f7fd966095c777ac86654e59f29ede795fa016b2')

## Other functionality testing

### Updating an authetication center to an enrollment center

In [90]:
accessControlContract.functions.updateNodeStatus(postOfficeAddress,                     # adress of the post office
                                                 True,                          # Authentication Center
                                                 True                           # Enrollment Center
                                                 ).transact({'from':Main_Enrollment_Center})

HexBytes('0x4a8e8bf04ba14e68b353706274da7c7b85ba7e443897c791344a88c16f75d457')

### Enroll a subject from the updated center

In [91]:
accessControlContract.functions.setSubject(4,                         # ID
                                           11110,                     # hx      # Random Numbe
                                           1110                       # delta   # Random Number
                                           ).transact({'from': postOfficeAddress}) # Enrolled by  (Post Office)

HexBytes('0x35e36a67ea3a8fc318694d36fa48926730bd2ca6d2ab42146220ba81fa453223')

## Preventing White-hat Hacker from unauthorization acces


### Gain access to the data

In [92]:
try:
    result = accessControlContract.functions.getSubject(3).call({'from': whiteHatHackerAddress})
    # Process the result here
except TransactionFailed as e:
    print(f"Transaction failed: {e}")
except NameError as e:
    print(f"Error: {e}")

Transaction failed: execution reverted: Caller is not an authorized Authentication Center


In [93]:
try:
    result = accessControlContract.functions.getNode(whiteHatHackerAddress).call({'from':whiteHatHackerAddress})
    # Process the result here
except TransactionFailed as e:
    print(f"Transaction failed: {e}")
except NameError as e:
    print(f"Error: {e}")

Transaction failed: execution reverted: Caller is not an Enrollment Center


### Enroll subject from non-enroll center

In [94]:
try:
    result = accessControlContract.functions.setSubject(5,10001,1000).transact({'from': whiteHatHackerAddress})
    # Process the result here
except TransactionFailed as e:
    print(f"Transaction failed: {e}")
except NameError as e:
    print(f"Error: {e}")

Transaction failed: execution reverted: Caller is not an Enrollment Center


# Interacting with the smart contract

In [82]:
# ID =  Web3.keccak(text="BLZNRS")
# hx =  Web3.to_bytes(10110)
# delta =  Web3.to_bytes(1001)
print("Enrollmetn Started") # BLZNRS Codice Fiscale
ID = input("Please enter ID: ")
ID_int = int(ID)
hx = input("Please enter Hx: ")
hx_int = int(hx)
delta = input("Please enter delta: ")
delta_int = int(delta)
accessControlContract.functions.setSubject(ID_int,                         # ID
                                           hx_int,                     # hx      # Random Number
                                           delta_int                       # delta   # Random Number
                                           ).transact({'from': Main_Enrollment_Center}) # Enrolled by BB01

print("Authentication started")
ID = input("Please enter ID: ")
ID_int = int(ID)
data = accessControlContract.functions.getSubject(ID_int                                    # ID to be retrived
                                           ).call({'from':Main_Enrollment_Center}) # Call from BB01

delta = data[2]
hx = data[2]
print("Delta:\t", delta, "\nhx:\t",hx)

Enrollmetn Started
Please enter ID: 5
Please enter Hx: 11011
Please enter delta: 10100
Authentication started
Please enter ID: 5
Delta:	 10100 
hx:	 10100
